In [125]:
from tkinter import Tk
from tkinter.filedialog import askopenfilename
import pandas as pd
from math import floor, ceil

fname = askopenfilename()

In [126]:
class annot:
    def __init__(self): #add fname, and calcium options
        self.channels = []
        self.annots = []
        # initilize dataframe of channels

    def add_ch(self, ch):
        self.channels.append(ch)

    def add_an(self, an):
        self.annots.append(an)

In [127]:
esr530 = annot()
stamp_count = [99] * 6747
t_offset = 0.256 #you should probably set this to 0.
vid_time = 263634/585.937500 - t_offset #frame count/frame rate for total video time in seconds
behav_dict = {}

with open(fname) as file:
    col = False
    behav_list = stamp_count[:] #initilizes list of 0 (no behavior)
    for line in file:
        #if 'Annotation stop frame' in line:

        if 'List of channels' in line:
            ch = file.readline()
            while ch != '\n':
                esr530.add_ch(ch[0:-1])
                ch = file.readline()

        elif 'List of annotations' in line:
            an = file.readline()
            while an != '\n':
                esr530.add_an(an[0:-1])
                an = file.readline()

        elif '-'*10 in line:
            if col:
                behav_dict[col] = behav_list
                behav_list = stamp_count[:]
            
            col = line.strip('-\n')
        
        elif '>' in line:
            bi = esr530.annots.index(line.strip('>\n'))
            file.readline() #need to skip a line
            t = file.readline()
            while t != '\n':
                start_frame = (float(t.split()[0]) - t_offset) / vid_time * (len(stamp_count)-1)
                if start_frame < 0:
                    start_frame = 0
                stop_frame = (float(t.split()[1]) - t_offset) / vid_time * (len(stamp_count)-1)
                if stop_frame > len(stamp_count)-1:
                    stop_frame = len(stamp_count)-1
                for i in range(floor(start_frame), ceil(stop_frame)):
                    behav_list[i]=bi
                t = file.readline()
    behav_dict[col] = behav_list
    behav_list = stamp_count[:]

    df = pd.DataFrame(behav_dict)

In [129]:
dfb = pd.DataFrame(behav_dict)
df_key = pd.DataFrame(esr530.annots)

dfb.to_csv('esr530-t6_annot.csv', index=False)
df_key.to_csv('esr530-t6_key.csv', header=False)

with open('esr530-t6_key.csv', 'a') as c: # probably a more readable alternative to this
    c.write('99,none')